In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
base_path = '../input/tabular-playground-series-sep-2021/'

In [ ]:
train = pd.read_csv(base_path + 'train.csv')
train

In [ ]:
test = pd.read_csv(base_path + 'test.csv')
test

In [ ]:
train_claim = pd.DataFrame(train.claim).reset_index()
train_claim.drop('index', axis=1, inplace=True)

In [ ]:
cate_base_path = '../input/train-cate/'

In [ ]:
cate_train = pd.read_csv(cate_base_path + 'X_after_diet.csv')
cate_train

In [ ]:
cate_train.drop('Unnamed: 0', axis=1, inplace=True)
cate_train

In [ ]:
total_train = pd.concat([cate_train, train_claim], axis=1)
total_train

In [ ]:
del train
del cate_train
del train_claim

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
# def modelfit(skf, alg, dtrain, features, train_oof):
#     for fold, (train_idx, valid_idx) in enumerate(skf.split(dtrain[features], dtrain.claim)):
#         X_train, X_valid = dtrain[features].iloc[train_idx], dtrain[features].iloc[valid_idx]
#         y_train, y_valid = dtrain.claim.iloc[train_idx], dtrain.claim.iloc[valid_idx]
        
#         alg.fit(X_train, y_train, verbose=0)
# #         dtrain_predictions = alg.predict(X_valid)
#         dtrain_predprob = alg.predict_proba(X_valid)[:, 1]
#         train_oof[valid_idx] = dtrain_predprob

#         print("\nModel Report")
#         print("Fold %d Training AUC : %.8f"% (fold, roc_auc_score(y_valid, dtrain_predprob)))
              
#     print("Training AUC : %.8f" % roc_auc_score(dtrain.claim, train_oof))

In [ ]:
# features = [x for x in total_train.columns if x not in ['claim']]

# folds = 5

# xgb1 = XGBClassifier(
#     learning_rate = 0.1,
#     n_estimators = 1000,
#     max_depth = 5,
#     min_child_weight = 1,
#     gamma = 0,
#     subsample = 0.8,
#     colsample_bytree = 0.8,
#     objective = 'binary:logistic',
#     nthread = -1,
#     scale_pos_weight = 1,
#     random_state = 42,
#     verbosity = 0,
#     tree_method='gpu_hist'
# )

# skf1 = StratifiedKFold(n_splits = folds, shuffle = True, random_state = 42)

# train_oof = np.zeros((957919,))

# modelfit(skf1, xgb1, total_train, features, train_oof)

In [ ]:
def model_tunning_fit(alg, dtrain, features, use_train = True, cv_folds = 5):
    
    if use_train:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[features].values, label=dtrain['claim'].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round = alg.get_params()['n_estimators'], nfold = cv_folds, metrics='auc')
        alg.set_params(n_estimators=cvresult.shape[0])
        print(alg)
    
    alg.fit(dtrain[features], dtrain['claim'], verbose=0)
    dtrain_predictions = alg.predict(dtrain[features])
    dtrain_predprob = alg.predict_proba(dtrain[features])[:, 1]

    print("\nModel Report")
    print(f"Fold Training AUC : ", roc_auc_score(dtrain['claim'], dtrain_predictions))

In [ ]:
features = [x for x in total_train.columns if x not in ['claim']]

xgb2 = XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 5000,
    max_depth = 3,
    min_child_weight = 1,
    gamma = 0.0,
    subsample = 0.71,
    colsample_bytree = 0.9,
    objective = 'binary:logistic',
    nthread = -1,
    scale_pos_weight = 1,
    reg_alpha = 0.1,
    random_state = 42,
    verbosity = 0,
    tree_method='gpu_hist'
)

# folds = 10

# skf1 = StratifiedKFold(n_splits = folds, shuffle = True, random_state = 42)
# gamma = 0.0 or 0.1 subsample = 0.9 or 0.76
# model_tunning_fit(xgb1, total_train, features)

In [ ]:
# model_tunning_fit(xgb2, total_train, features)

In [ ]:
# modelfit(skf1, xgb2, total_train, features, train_oof)

In [ ]:
param_test1 = {
 'max_depth':range(3,10,3),
 'min_child_weight':range(1,6,2)
}

skf2 = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

gsearch1 = GridSearchCV(estimator = xgb2, param_grid = param_test1, scoring='roc_auc',n_jobs=-1,iid=False, cv=skf2, verbose=10)
gsearch1.fit(total_train[features],total_train['claim'])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = xgb2, param_grid = param_test2, scoring='roc_auc', n_jobs=-1, iid=False, cv=skf2)
gsearch2.fit(total_train[features],total_train['claim'])
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = xgb2, param_grid = param_test3, scoring='roc_auc', n_jobs=-1, iid=False, cv=skf2, verbose=10)
gsearch3.fit(total_train[features],total_train['claim'])
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {
 'subsample':[i/100.0 for i in range(40,80)],
}

gsearch4 = GridSearchCV(estimator = xgb2, param_grid = param_test4, scoring='roc_auc', n_jobs=-1, iid=False, cv=skf2, verbose=10)
gsearch4.fit(total_train[features],total_train['claim'])
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

gsearch5 = GridSearchCV(estimator = xgb2, param_grid = param_test5, scoring='roc_auc', n_jobs=-1, iid=False, cv=skf2, verbose=10)
gsearch5.fit(total_train[features],total_train['claim'])
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_